Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [29]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [30]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [31]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [32]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [38]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

In [101]:
batch_size = 128
nodes = 1024
reg_coeff = 6*1e-2
reg_coeff = 6*1e-3
#reg_coeff = 6*1e-4
#reg_coeff = 6*1e-5
#reg_coeff = 6*1e-6
reg_coeff = 6*1e-20


graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  w1 = weight_variable([image_size * image_size, nodes])
  b1 = bias_variable([nodes])
  w2 = weight_variable([nodes, num_labels])
  b2 = bias_variable([num_labels])
    
  # Hidden Layer
  h1 = tf.nn.relu(tf.matmul(tf_train_dataset, w1) + b1)

  # L2 regularization for the fully connected parameters.
  regularizers = (tf.nn.l2_loss(w2) +
              tf.nn.l2_loss(w1))
  
  # Training computation.
  logits = tf.matmul(h1, w2) + b2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Add the regularization term to the loss.
  loss += reg_coeff * regularizers
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, w1) + b1), w2) + b2)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, w1) + b1), w2) + b2)

In [102]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size),:]
    batch_labels = train_labels[offset:(offset + batch_size),:]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.380601
Minibatch accuracy: 10.2%
Validation accuracy: 34.4%
Minibatch loss at step 500: 0.548480
Minibatch accuracy: 82.8%
Validation accuracy: 85.0%
Minibatch loss at step 1000: 0.637532
Minibatch accuracy: 85.9%
Validation accuracy: 83.4%
Minibatch loss at step 1500: 0.685136
Minibatch accuracy: 82.8%
Validation accuracy: 85.5%
Minibatch loss at step 2000: 0.527091
Minibatch accuracy: 86.7%
Validation accuracy: 87.4%
Minibatch loss at step 2500: 0.311658
Minibatch accuracy: 93.8%
Validation accuracy: 87.9%
Minibatch loss at step 3000: 0.291516
Minibatch accuracy: 89.8%
Validation accuracy: 88.1%
Test accuracy: 93.8%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [99]:
batch_size = 8
nodes = 1024
reg_coeff = 6*1e-2
reg_coeff = 6*1e-3
#reg_coeff = 6*1e-4
#reg_coeff = 6*1e-5
#reg_coeff = 6*1e-6
reg_coeff = 6*1e-20


graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  w1 = weight_variable([image_size * image_size, nodes])
  b1 = bias_variable([nodes])
  w2 = weight_variable([nodes, num_labels])
  b2 = bias_variable([num_labels])
    
  # Hidden Layer
  h1 = tf.nn.relu(tf.matmul(tf_train_dataset, w1) + b1)

  # L2 regularization for the fully connected parameters.
  regularizers = (tf.nn.l2_loss(w2) +
              tf.nn.l2_loss(w1))
  
  # Training computation.
  logits = tf.matmul(h1, w2) + b2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Add the regularization term to the loss.
  loss += reg_coeff * regularizers
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, w1) + b1), w2) + b2)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, w1) + b1), w2) + b2)

In [100]:
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Restrict offset to only three values
    offset = (step%100)*batch_size
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size),:]
    batch_labels = train_labels[offset:(offset + batch_size),:]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.226016
Minibatch accuracy: 12.5%
Validation accuracy: 11.1%
Minibatch loss at step 500: 362312.937500
Minibatch accuracy: 50.0%
Validation accuracy: 45.4%
Minibatch loss at step 1000: 659009664.000000
Minibatch accuracy: 37.5%
Validation accuracy: 39.7%
Minibatch loss at step 1500: 4470232055808.000000
Minibatch accuracy: 50.0%
Validation accuracy: 41.1%
Minibatch loss at step 2000: 23520224460406784.000000
Minibatch accuracy: 25.0%
Validation accuracy: 45.8%
Minibatch loss at step 2500: 47605897995445338112.000000
Minibatch accuracy: 62.5%
Validation accuracy: 46.0%
Minibatch loss at step 3000: 337476877789037928120320.000000
Minibatch accuracy: 50.0%
Validation accuracy: 43.5%
Test accuracy: 46.8%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [91]:
batch_size = 8
nodes = 1024
reg_coeff = 6*1e-20

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(None, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(None, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  w1 = weight_variable([image_size * image_size, nodes])
  b1 = bias_variable([nodes])
  w2 = weight_variable([nodes, num_labels])
  b2 = bias_variable([num_labels])
  
  # Hidden Layer
  h1 = tf.nn.relu(tf.matmul(tf_train_dataset, w1) + b1)

  # Dropout
  #keep_prob = tf.placeholder(tf.float32)
  h_drop = tf.nn.dropout(h1, 0.5)
  
  # Training computation.
  logits = tf.matmul(h_drop, w2) + b2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, w1) + b1), w2) + b2)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, w1) + b1), w2) + b2)

In [92]:
batch_size = 8

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Restrict offset to only three values
    offset = (step%100)*batch_size
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size),:]
    batch_labels = train_labels[offset:(offset + batch_size),:]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.866057
Minibatch accuracy: 25.0%
Validation accuracy: 16.4%
Minibatch loss at step 500: nan
Minibatch accuracy: 37.5%
Validation accuracy: 10.0%
Minibatch loss at step 1000: nan
Minibatch accuracy: 37.5%
Validation accuracy: 10.0%
Minibatch loss at step 1500: nan
Minibatch accuracy: 37.5%
Validation accuracy: 10.0%
Minibatch loss at step 2000: nan
Minibatch accuracy: 37.5%
Validation accuracy: 10.0%
Minibatch loss at step 2500: nan
Minibatch accuracy: 37.5%
Validation accuracy: 10.0%
Minibatch loss at step 3000: nan
Minibatch accuracy: 37.5%
Validation accuracy: 10.0%
Test accuracy: 10.0%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [104]:
batch_size = 128
nodes1 = 1024
nodes2 = 1024
reg_coeff = 8*1e-5

graph = tf.Graph()
with graph.as_default():
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(None, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(None, num_labels))
  
  # Variables.
  w1 = weight_variable([image_size * image_size, nodes1])
  b1 = bias_variable([nodes1])
  w2 = weight_variable([nodes1, nodes2])
  b2 = bias_variable([nodes2])
  w3 = weight_variable([nodes2, num_labels])
  b3 = bias_variable([num_labels])
  prob = tf.placeholder(tf.float32)
    
  # Hidden Layer1
  h1 = tf.nn.relu(tf.matmul(tf_train_dataset, w1) + b1)
  # Dropout1
  h_d1 = tf.nn.dropout(h1, prob)

  # Hidden Layer2
  h2 = tf.nn.relu(tf.matmul(h_d1, w2) + b2)
  # Dropout2
  h_d2 = tf.nn.dropout(h2, prob)   
    
  # L2 regularization for the fully connected parameters.
  regularizers = (tf.nn.l2_loss(w2) + tf.nn.l2_loss(w1) +
    tf.nn.l2_loss(w3))
  
  # Training computation.
  logits = tf.matmul(h_d2, w3) + b3
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Add the regularization term to the loss.
  loss += reg_coeff * regularizers
    
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 100000, 0.96, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  correct_prediction = tf.equal(tf.argmax(train_prediction,1), tf.argmax(tf_train_labels,1))
  accurate = tf.reduce_mean(tf.cast(correct_prediction, "float"))*100

In [105]:
num_steps = 2501

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    #print(step)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size),:]
    batch_labels = train_labels[offset:(offset + batch_size),:]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, prob: 0.5}
    _, l, _ = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accurate.eval(feed_dict = {
                    tf_train_dataset : batch_data, tf_train_labels : batch_labels, prob: 1}))
      print("Validation accuracy: %.1f%%" % accurate.eval(feed_dict = {
                tf_train_dataset : valid_dataset, tf_train_labels : valid_labels, prob: 1}))
  print("Test accuracy: %.1f%%" % accurate.eval(feed_dict = {
            tf_train_dataset : test_dataset, tf_train_labels : test_labels, prob: 1}))

Initialized
Minibatch loss at step 0: 13.983371
Minibatch accuracy: 20.3%
Validation accuracy: 17.0%
Minibatch loss at step 500: 1.551496
Minibatch accuracy: 78.1%
Validation accuracy: 79.2%
Minibatch loss at step 1000: 1.408400
Minibatch accuracy: 84.4%
Validation accuracy: 81.6%
Minibatch loss at step 1500: 1.587572
Minibatch accuracy: 77.3%
Validation accuracy: 81.9%
Minibatch loss at step 2000: 1.236686
Minibatch accuracy: 82.8%
Validation accuracy: 83.2%
Minibatch loss at step 2500: 1.047308
Minibatch accuracy: 91.4%
Validation accuracy: 82.9%
Test accuracy: 90.0%


In [ ]:
batch_size = 128
nodes1 = 1024
nodes2 = 1024
reg_coeff = 8*1e-5

Initialized
Minibatch loss at step 0: 16.019089
Minibatch accuracy: 29.7%
Validation accuracy: 23.5%
Minibatch loss at step 500: 1.542730
Minibatch accuracy: 79.7%
Validation accuracy: 79.6%
Minibatch loss at step 1000: 1.298317
Minibatch accuracy: 86.7%
Validation accuracy: 82.2%
Minibatch loss at step 1500: 1.367436
Minibatch accuracy: 81.2%
Validation accuracy: 82.7%
Minibatch loss at step 2000: 1.195550
Minibatch accuracy: 83.6%
Validation accuracy: 84.1%
Minibatch loss at step 2500: 0.990642
Minibatch accuracy: 93.8%
Validation accuracy: 84.2%
Minibatch loss at step 3000: 0.951867
Minibatch accuracy: 90.6%
Validation accuracy: 84.1%
Test accuracy: 90.9%
    
    

batch_size = 128
nodes1 = 1024
nodes2 = 1024
reg_coeff = 8*1e-6

Initialized
Minibatch loss at step 0: 12.488296
Minibatch accuracy: 21.9%
Validation accuracy: 14.6%
Minibatch loss at step 500: 1.008491
Minibatch accuracy: 74.2%
Validation accuracy: 77.3%
Minibatch loss at step 1000: 0.717178
Minibatch accuracy: 84.4%
Validation accuracy: 80.6%
Minibatch loss at step 1500: 0.868731
Minibatch accuracy: 79.7%
Validation accuracy: 81.2%
Minibatch loss at step 2000: 0.889091
Minibatch accuracy: 77.3%
Validation accuracy: 81.3%
Minibatch loss at step 2500: 0.605024
Minibatch accuracy: 90.6%
Validation accuracy: 81.6%
Minibatch loss at step 3000: 0.518629
Minibatch accuracy: 90.6%
Validation accuracy: 82.7%
Test accuracy: 89.6%